In [4]:
pip install xlrd>=2.0.1


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import json
import os
import numpy as np

def load_data(file_path):
    """
    Carica un file CSV, XLS, JSON o JSONL e restituisce un DataFrame con le colonne 'name', 'category', 'address', 'city', 'country'.
    """
    encodings = ['utf-8', 'ISO-8859-1', 'latin1']
    
    for encoding in encodings:
        try:
            if file_path.endswith('.csv'):
                # Prova a leggere il file CSV con encoding
                df = pd.read_csv(file_path, encoding=encoding)
            elif file_path.endswith(('.xls', '.xlsx')):
                # Gestione dei file Excel
                try:
                    df = pd.read_excel(file_path)
                except Exception as e:
                    print(f"Errore con il file {file_path} (Excel) usando encoding {encoding}: {e}")
                    continue
            elif file_path.endswith('.json'):
                # Gestione dei file JSON
                with open(file_path, 'r', encoding=encoding) as f:
                    data = json.load(f)
                
                # Verifica la struttura dei dati nel JSON
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                elif isinstance(data, dict):
                    # In caso di dizionario, estrai i dati necessari e crea il DataFrame
                    df = pd.DataFrame([data])
                else:
                    # Se i dati non sono né una lista né un dizionario, non possiamo trattarli correttamente
                    raise ValueError(f"Formato del JSON non supportato in {file_path}")
                
            elif file_path.endswith('.jsonl'):
                # Gestione dei file JSONL
                df = pd.read_json(file_path, lines=True)
            else:
                raise ValueError("Formato non supportato")
            
            # Inizializzazione dei valori come NaN
            name_value = np.nan
            category_value = np.nan
            address_value = np.nan
            city_value = np.nan
            country_value = np.nan

            # Trova il primo campo per "name", "category", "address", "city" e "country"
            name_columns = ["Name", "name", "Company", "BRAND NAME"]
            category_columns = ["category", "Area of Activity", "company_business", "industry", "CATEGORY", "Industry"]
            address_columns = ["address", "Address Name", "Address"]
            city_columns = ["city","City","headquarters_region_city", "headquarters", "location", "Headquarter"]
            country_columns = ["country", "Country", "headquarters_country", "headquarters","Headquarter"]

            for col in name_columns:
                if col in df.columns:
                    name_value = df[col].astype(str).str.strip()  # Mantieni come nel file
                    break

            for col in category_columns:
                if col in df.columns:
                    category_value = df[col]
                    break

            for col in address_columns:
                if col in df.columns:
                    address_value = df[col]
                    if col == "address":
                        address_value = address_value.str.split(',').str[0]  # Primo valore prima della virgola
                    break

            for col in city_columns:
                if col in df.columns:
                    # Se city è in address (contenuto in "location_columns" come "address")
                    city_value = df[col]
                    if col == "address":
                        address_split = city_value.str.split(',')
                        if address_split.str.len() > 2:  # Se ci sono almeno 3 valori separati da virgola
                            city_value = address_split.str[2]  # Prendi il terzo valore
                        else:
                            city_value = np.nan  # Se non ci sono abbastanza valori, lascia NaN
                    elif col in ["Headquarter", "headquarters"]:
                        city_value = city_value.str.split(',').str[1]  # Primo valore prima della virgola
                    break

            for col in country_columns:
               if col in df.columns:
                country_value = df[col]
                if col in ["headquarters", "Headquarter"]:
                   country_value = country_value.str.split(',').str[2]  # Prendi il terzo valore dopo la virgola
                break

            # Crea il DataFrame con i valori estratti
            result_df = pd.DataFrame({
                "name": name_value, 
                "category": category_value, 
                "address": address_value, 
                "city": city_value, 
                "country": country_value
            }, index=range(len(df)))
            
            return result_df
        
        except Exception as e:
            print(f"Errore con il file {file_path} usando encoding {encoding}: {e}")
    
    # Se non riusciamo a caricare il file
    print(f"Impossibile caricare il file {file_path} con i codici di encoding disponibili.")
    return pd.DataFrame(columns=["name", "category", "address", "city", "country"])

# Cartella contenente i file
source_folder = "sources/"

# Ottieni la lista dei file nella cartella source
file_list = [os.path.join(source_folder, file) for file in os.listdir(source_folder) 
             if file.endswith(('.csv', '.xls', '.xlsx', '.json', '.jsonl'))]

# Separare i file in due gruppi
wissel_files = [file for file in file_list if os.path.basename(file) in [
    "wissel-rappresentanti-ariregister.rik.ee.csv", "wissel-partners-ariregister.rik.ee.csv"]]
other_files = [file for file in file_list if file not in wissel_files]

# Creare due DataFrame separati
wissel_df = pd.concat([load_data(file) for file in wissel_files], ignore_index=True)
other_df = pd.concat([load_data(file) for file in other_files], ignore_index=True)

# Salvare i risultati in due CSV separati
wissel_df.to_csv("output_wissel.csv", index=False, encoding='utf-8')
other_df.to_csv("output_other.csv", index=False, encoding='utf-8')

print("Schema unificato salvato in 'output_wissel.csv' e 'output_other.csv'")


Errore con il file sources/AmbitionBox.csv usando encoding utf-8: 'utf-8' codec can't decode byte 0xe9 in position 3: invalid continuation byte
Schema unificato salvato in 'output_wissel.csv' e 'output_other.csv'
